<a href="https://colab.research.google.com/github/victoralcantara75/CNN-FLOWER-CLASSIFY/blob/master/CNN_FLOWER_CLASSIFY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CNN PARA CLASSIFICAÇÃO DE FLORES
## SIN493 - DEEP LEARNING PARA VISÃO COMPUTACIONAL

##PYTORCH E CUDA

In [0]:
#http://pytorch.org/

from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

fastai 1.0.42 has requirement torch>=1.0.0, but you'll have torch 0.4.1 which is incompatible.


In [0]:
import numpy as np

# verificando se o CUDA está disponível
GPUavailable = torch.cuda.is_available()

if GPUavailable:
    print('Treinamento em GPU!')
else:
    print('Treinamento em CPU!')

Treinamento em GPU!


##DATASET

In [0]:
from torchvision import datasets
import torchvision.transforms as transforms

!wget -cq https://github.com/udacity/pytorch_challenge/raw/master/cat_to_name.json
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!unzip -qq flower_data.zip

In [54]:
# %matplotlib inline
# import glob
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# import numpy as np

# PATH = '../train/'
# listing = glob.glob(f'{PATH}*jpg')[:10];
# listing

# for images in listing :
#   img = mpimg.inread(images);
#   plt.figure();
#   imgplot = plt.imshow(img)

[]

In [0]:
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [0]:
# APLICANDO AS TRANSFORMACOES NECESSARIAS E O DATA AUGMENTATION

transformacao = {'train': 
                   transforms.Compose([ 
                   transforms.RandomRotation(10),      #ROTAÇÃO DAS IMAGENS
                   transforms.RandomResizedCrop(224),  #INPUT SIZE 224X224
                   transforms.RandomHorizontalFlip(),  #ESPELHAMENTO
                   transforms.ToTensor(),
                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ]),
                'valid': 
                   transforms.Compose([ 
                   transforms.Resize(256),
                   transforms.CenterCrop(224),
                   transforms.ToTensor(),
                   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ])
}

In [0]:
import os

image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), transformacao[x])
    for x in ['train', 'valid']
}

batch_size =4 

dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, 
        shuffle=True, num_workers=4)
    for x in ['train', 'valid']
}

In [0]:
#MAPEANDO OS LABELS
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

##TREINO E CLASSIFICACAO VGG

In [0]:
import torchvision.models as models

#criando o modelo de vgg 16 pre treinada
VGG = models.vgg16(pretrained=True)
#visualizacao da arquitetura utilizada vgg16
VGG

# movendo os tensors para GPU se o CUDA estiver disponível
if GPUavailable:
    VGG.cuda()


In [0]:
import torch.nn as nn
import torch.optim as optim

# Especificando a loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# Definindo o Otimizador
optimizer = optim.SGD(VGG.parameters(), lr=0.01)

In [72]:
#TREINAMENTO 

epocas = 30

for epocas in range (0, epocas):

  treino_loss = 0.0
  validacao_loss = 0.0
  
  VGG.train()
  for data, target in image_datasets:
    
        if GPUavailable:
            data, target = data.cuda(), target.cuda()
          
        optimizer.zero_grad()
        output = VGG(data)
        loss = criterion(output, target) #CALCULAR O LOSS
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0) #ATUALIZAR O TRAINING LOSS
        
        
  VGG.eval()
  for data, target in dataloaders:
    
      if GPUavailable:
          data, target = data.cuda(), target.cuda()
          
      output = VGG(data)
      loss = criterion(output, target)
      valid_loss += loss.item()*data.size(0)      

      
  # Calcular as perdas médias (losses)
  treino_loss = train_loss/len(train_loader.dataset)
  validacao_loss = valid_loss/len(valid_loader.dataset)
  
  # Impressão das estatísticas de treinamento/validação
  print('Época: {} \tTreinamento Loss: {:.6f} \tValidacao Loss: {:.6f}'.format(epoch, train_loss, valid_loss))
    
  # Salvar o modelo se a perda da valiaçaõ tiver reduzido
  if valid_loss <= valid_loss_min:
      print('Validacao loss decrementou ({:.6f} --> {:.6f}).  Salvando o modelo ...'.format(
      valid_loss_min,
      valid_loss))
      torch.save(model.state_dict(), 'model_cifar.pt')
      valid_loss_min = valid_loss

ValueError: ignored